In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split
# import keras_efficientnet_v2
from tensorflow.keras.applications import efficientnet
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from tensorflow.keras import metrics, optimizers, Sequential, activations, callbacks
from sklearn.metrics import f1_score, classification_report, confusion_matrix, accuracy_score
import keras.backend as K
from tensorflow import keras
import warnings
warnings.filterwarnings('ignore')

In [27]:
train_dir = r"asl_data/asl_alphabet_train/asl_alphabet_train"
test_dir = r"data_47/data_47/asl_alphabet_train"

In [28]:
def load_df(dir_path):
    df = {
        'filename': [],
        'label': []
    }
    classes = os.listdir(dir_path)
    for c in classes:
        path = dir_path + '/' + c

        img_paths = os.listdir(path)

        for img_path in img_paths:
            df['filename'].append(c + '/' + img_path)
            df['label'].append(c)
        
    df = pd.DataFrame(df)
    return df

train_df = load_df(train_dir)
test_df = load_df(test_dir)

In [31]:
train_df.sample(5)

,filename,label
28855,B/B1700.jpg,B
29304,B/B2315.jpg,B
33276,M/M2153.jpg,M
22825,W/W2109.jpg,W
29053,B/B2393.jpg,B


In [32]:
X = train_df['filename']
y = train_df['label']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, random_state=1, test_size=0.2)
print(X_train.shape)
print(X_valid.shape)

(27081,)
(6771,)


In [33]:
train = pd.concat([X_train, y_train], axis=1)
valid = pd.concat([X_valid, y_valid], axis=1)
test = test_df

In [34]:
train['label'].value_counts()

D    1042
H    1042
O    1042
X    1042
T    1042
W    1042
C    1042
N    1042
G    1042
E    1042
V    1042
S    1042
R    1042
U    1042
K    1042
J    1041
Y    1041
B    1041
Q    1041
Z    1041
L    1041
F    1041
I    1041
A    1041
M    1041
P    1041
Name: label, dtype: int64

In [35]:
valid['label'].value_counts()

A    261
L    261
M    261
Z    261
Y    261
F    261
Q    261
P    261
I    261
B    261
J    261
X    260
T    260
C    260
D    260
E    260
S    260
R    260
W    260
O    260
K    260
V    260
U    260
G    260
H    260
N    260
Name: label, dtype: int64

In [36]:
test['label'].value_counts()

J          47
H          47
X          47
R          47
B          47
L          47
P          47
E          47
V          47
W          47
F          47
A          47
space      47
S          47
T          47
Z          47
nothing    47
del        47
U          47
Y          47
C          47
D          47
G          47
O          47
N          47
I          47
Q          47
K          47
M          47
Name: label, dtype: int64

In [37]:
IMAGE_SIZE = (224, 224)

datagen = ImageDataGenerator(
    rescale=1./255,
#     rotation_range=40,
    zoom_range=0.2,
    width_shift_range=0.05,
    height_shift_range=0.05,
#     brightness_range=[0.5, 1.0],
)

train_generator = datagen.flow_from_dataframe(
    dataframe = train,
    directory = train_dir,
    x_col='filename',
    y_col='label',
    batch_size=32,
    # color_mode="rgb",
    seed=42,
    shuffle=True,
    class_mode='sparse',
    target_size=IMAGE_SIZE
)

valid_generator=datagen.flow_from_dataframe(
    dataframe=valid,
    directory=train_dir,
    x_col="filename",
    y_col="label",
    batch_size=32,
    # color_mode="rgb",
    seed=42,
    shuffle=True,
    class_mode="sparse",
    target_size=IMAGE_SIZE
)

test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
    dataframe=test,
    directory=test_dir,
    x_col="filename",
    y_col=None,
    batch_size=1,
    # color_mode="rgb",
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=IMAGE_SIZE
)

Found 27081 validated image filenames belonging to 26 classes.
Found 6771 validated image filenames belonging to 26 classes.
Found 1363 validated image filenames.


In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()
num_classes = 27

backbone = tf.keras.applications.EfficientNetB0(include_top=False)
backbone.trainable = False

inputs = Input(shape=IMAGE_SIZE + (3,), name='inputs')
x = backbone(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
# x = Dense(100, activation='relu')(x)
outputs = Dense(27, activation='softmax')(x)

model_efficientnet = tf.keras.Model(inputs=inputs, outputs=outputs, name='efficientnet_model')
model_efficientnet.summary()